In [120]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
%matplotlib inline
from pandas import read_excel
df_soc = pd.read_excel('soc.xlsx')#Social-demographical data (client ID, age, gender, tenure in a bank)
df_product = pd.read_excel('products.xlsx')#Products and actual balance (current account, saving account, mutual funds, overdraft, credit card, consumer loan)
df_inoutflow = pd.read_excel('inoutflow.xlsx')#Inflow/outflow on C/A, aggregated card turnover (monthly average over past 3 months)
df_sales = pd.read_excel('sales.xlsx')#Clients actual sales and revenues. Data is available for 60 % of the clients.

In [181]:
df_sales

,Client,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL
0,1094,0,1,0,0.000000,5.138571,0.000000
1,1484,1,0,1,25.609107,0.000000,8.722500
2,20,0,1,0,0.000000,3.822500,0.000000
3,1193,0,0,0,0.000000,0.000000,0.000000
4,142,0,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
964,1316,0,0,1,0.000000,0.000000,3.317500
965,905,1,1,1,1.761071,0.892857,13.166786
966,938,0,1,1,0.000000,4.928571,7.812143
967,434,0,0,0,0.000000,0.000000,0.000000


In [121]:
#Training dataset for MF product 
df_new=pd.merge(df_sales, df_inoutflow, on='Client', how='left')
df_new=pd.merge(df_new, df_soc, on='Client', how='left')
df_new['Sex']=df_new['Sex'].replace({'M':'0','F':'1'})
df_new['Age']=pd.qcut(df_new['Age'], 3, labels=["1", "2", "3"])
df_new['Tenure']=pd.qcut(df_new['Tenure'], 5, labels=["1", "2", "3","4","5"])
df_new['Age']=df_new['Age'].astype('O')
#R1=pd.crosstab(df_new['Revenue_MF'], df_new['Age'])
ss = StandardScaler()
df_new_scaled = pd.DataFrame(ss.fit_transform(df_new),columns = df_new.columns)
df_new_scaled

,Client,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,VolumeCred,VolumeCred_CA,TransactionsCred,...,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sex,Age,Tenure
0,0.601339,-0.498710,1.733243,-0.653528,-0.193693,0.137169,-0.465684,-0.295148,-0.308261,-0.726486,...,0.582980,0.055544,0.894542,1.321827,-0.376219,1.868005,-0.120342,-0.936840,-1.188848,-1.409488
1,1.441262,2.005175,-0.576953,1.530157,2.383072,-0.153403,0.661309,-0.076880,-0.026429,-0.101672,...,0.434387,-0.116772,0.405523,0.706264,-0.376219,1.270791,0.070719,-0.936840,0.032778,-1.409488
2,-1.711679,-0.498710,1.733243,-0.653528,-0.193693,0.062749,-0.465684,-0.222245,-0.214349,-0.257876,...,-0.441290,-0.198875,-0.572515,-0.524863,0.362242,-0.520851,-0.311403,-0.936840,-1.188848,1.419707
3,0.814550,-0.498710,-0.576953,-0.653528,-0.193693,-0.153403,-0.465684,-0.062746,-0.068740,2.709993,...,0.520545,-0.208045,2.361598,1.706554,1.469934,1.868005,-0.502464,-0.936840,0.032778,-1.409488
4,-1.448934,-0.498710,-0.576953,-0.653528,-0.193693,-0.153403,-0.465684,-0.180097,-0.159704,-0.726486,...,-0.473450,0.002126,-0.878151,-0.909591,-0.745450,-0.640294,-0.693525,1.067418,-1.188848,1.419707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,1.079449,-0.498710,-0.576953,1.530157,-0.193693,-0.153403,-0.037046,0.141924,0.246018,-0.257876,...,1.050835,0.423056,1.505815,1.937391,0.731473,2.465219,1.026024,1.067418,1.254405,0.712408
965,0.194300,2.005175,1.733243,1.530157,-0.016496,-0.102914,1.235534,-0.202716,-0.188910,-0.570283,...,-0.473450,-0.028387,-0.878151,-0.909591,-0.745450,-0.640294,-0.693525,-0.936840,0.032778,-1.409488
966,0.265370,-0.498710,1.733243,1.530157,-0.193693,0.125294,0.543686,-0.222913,-0.214991,-0.414079,...,0.191500,-0.179589,0.038759,0.244591,-0.006989,0.434691,0.261780,-0.936840,1.254405,-0.702189
967,-0.820069,-0.498710,-0.576953,-0.653528,-0.193693,-0.153403,-0.465684,-0.267250,-0.272238,-0.414079,...,-0.444902,-0.202335,-0.511387,-0.447918,0.731473,-0.401408,-0.693525,-0.936840,1.254405,0.712408


In [122]:
df_new_scaled=df_new_scaled.replace({np.NaN:'0'})
df_new_scaled = df_new_scaled[['Revenue_MF','Sale_MF','VolumeCred','VolumeCred_CA','TransactionsCred','TransactionsCred_CA','VolumeDebCash_Card','VolumeDebCashless_Card',
                              'TransactionsDeb_CA','TransactionsDebCash_Card','Sex','Age']]
df_new_scaled

,Revenue_MF,Sale_MF,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDebCash_Card,VolumeDebCashless_Card,TransactionsDeb_CA,TransactionsDebCash_Card,Sex,Age
0,-0.193693,-0.49871,-0.295148,-0.308261,-0.726486,-0.670345,-0.248888,0.58298,1.32183,-0.376219,-0.93684,-1.18885
1,2.38307,2.00517,-0.07688,-0.0264287,-0.101672,0.14305,-0.187535,0.434387,0.706264,-0.376219,-0.93684,0.0327784
2,-0.193693,-0.49871,-0.222245,-0.214349,-0.257876,-0.263648,-0.187535,-0.44129,-0.524863,0.362242,-0.93684,-1.18885
3,-0.193693,-0.49871,-0.0627459,-0.0687398,2.70999,3.39663,0.270564,0.520545,1.70655,1.46993,-0.93684,0.0327784
4,-0.193693,-0.49871,-0.180097,-0.159704,-0.726486,-0.670345,-0.31024,-0.47345,-0.909591,-0.74545,1.06742,-1.18885
...,...,...,...,...,...,...,...,...,...,...,...,...
964,-0.193693,-0.49871,0.141924,0.246018,-0.257876,-0.466997,0.147859,1.05084,1.93739,0.731473,1.06742,1.2544
965,-0.0164959,2.00517,-0.202716,-0.18891,-0.570283,-0.466997,-0.31024,-0.47345,-0.909591,-0.74545,-0.93684,0.0327784
966,-0.193693,-0.49871,-0.222913,-0.214991,-0.414079,-0.466997,-0.212076,0.1915,0.244591,-0.00698859,-0.93684,1.2544
967,-0.193693,-0.49871,-0.26725,-0.272238,-0.414079,-0.263648,-0.228437,-0.444902,-0.447918,0.731473,-0.93684,1.2544


## Linear Regression

In [123]:
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
Y=df_new_scaled.iloc[:,0].values
X=df_new_scaled.iloc[:,1:].values
train_X, test_X, train_Y, test_Y = train_test_split(X,Y, test_size=0.25,random_state=0)
lm = linear_model.LinearRegression() 
model=lm.fit(train_X, train_Y)
#y_pred = lm.predict(train_X)

In [124]:
# MSE test
y_pred_test = lm.predict(test_X)
RMSE=MSE(test_Y,y_pred_test)**(1/2)
RMSE

0.5722504264465518

In [125]:
# MSE train
y_pred_train= lm.predict(train_X)
RMSE=MSE(train_Y,y_pred_train)**(1/2)
RMSE

1.005915369556406

## Polynomial Regression

In [126]:
#Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures   
poly_reg = PolynomialFeatures(degree = 4) 
X_poly = poly_reg.fit_transform(train_X)  
X_test_poly = poly_reg.fit_transform(test_X)  
pol_reg = linear_model.LinearRegression() 
pol_reg.fit(X_poly, train_Y) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [127]:
X_test_poly = poly_reg.fit_transform(test_X)  
ypp = pol_reg.predict(X_test_poly)
RMSE=MSE(test_Y,ypp)**(1/2)
RMSE

307070693.3841712

In [111]:
yppt = pol_reg.predict(X_poly)
RMSE=MSE(train_Y,yppt)**(1/2)
RMSE

0.00013530712933331352

In [173]:
# Get the 40% of missing data from df_soc
df_40=pd.merge(df_soc, df_inoutflow, on='Client', how='left')
df_40_1=pd.merge(df_40, df_product, on='Client', how='left')
df_40_1['Sex']=df_40_1['Sex'].replace({'M':'0','F':'1'})
df_40_1 = df_40_1[['Sex','Client','VolumeCred','VolumeCred_CA','TransactionsCred','TransactionsCred_CA','VolumeDeb','VolumeDeb_CA','VolumeDebCash_Card','VolumeDebCashless_Card','VolumeDeb_PaymentOrder','TransactionsDeb','TransactionsDeb_CA','TransactionsDebCash_Card','TransactionsDebCashless_Card','TransactionsDeb_PaymentOrder','Age','Tenure']]
#df_train_MF=df_train_MF.dropna()
df_40_2=pd.concat([df_40_1, df_sales])
df_40_f=df_40_2.drop_duplicates(subset='Client', keep=False)
df_40_f = df_40_f[['Sale_MF','VolumeCred','VolumeCred_CA','TransactionsCred','TransactionsCred_CA','VolumeDebCash_Card','VolumeDebCashless_Card',
                             'TransactionsDeb_CA','TransactionsDebCash_Card','Sex','Age']]
df_40_f=df_40_f.replace({np.NaN:0})
df_40_f


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,Sale_MF,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDebCash_Card,VolumeDebCashless_Card,TransactionsDeb_CA,TransactionsDebCash_Card,Sex,Age
0,0.0,789.129643,738.230000,4.0,3.0,178.571429,0.000000,7.0,1.0,0,21.0
6,0.0,1033.496071,778.370000,8.0,6.0,89.285714,0.000000,10.0,2.0,0,37.0
9,0.0,435.682143,435.682143,2.0,2.0,125.000000,70.842143,10.0,4.0,1,19.0
10,0.0,132.158929,132.158929,3.0,3.0,0.000000,0.000000,1.0,0.0,0,61.0
13,0.0,570.157857,570.157857,2.0,2.0,71.428571,224.146429,17.0,2.0,1,41.0
...,...,...,...,...,...,...,...,...,...,...,...
1598,0.0,1406.490714,934.384643,7.0,5.0,0.000000,0.000000,5.0,0.0,1,33.0
1600,0.0,178.579643,178.579643,2.0,2.0,107.142857,0.000000,3.0,2.0,0,65.0
1608,0.0,811.332857,811.332857,3.0,3.0,160.714286,65.421429,12.0,2.0,0,68.0
1610,0.0,469.179643,469.179643,3.0,3.0,178.571429,11.414286,12.0,1.0,0,31.0


In [176]:
Prediction=lm.predict(df_40_f)
Revenue_MF_Prediction=pd.DataFrame(Prediction)
Revenue_MF_Prediction=Revenue_MF_Prediction.rename(columns={0:'Revenue_MF_Prediction'})
Revenue_MF_Prediction=pd.merge(df_40_f,Revenue_MF_Prediction,right_index=True,left_index=True,how='right')
rmp = pd.merge(Revenue_MF_Prediction, df_soc,right_index=True,left_index=True,how='left')
rmp=rmp[['Client','Revenue_MF_Prediction']]
spf = pd.read_pickle('spf.pickle')
Sales_Revenue_MF_Fprediction = pd.merge(spf, rmp, on = 'Client', how = 'left')
Sales_Revenue_MF_Fprediction=Sales_Revenue_MF_Fprediction[Sales_Revenue_MF_Fprediction.Revenue_MF_Prediction>0]
Sales_Revenue_MF_Fprediction=Sales_Revenue_MF_Fprediction.sort_values(by='Revenue_MF_Prediction', ascending=False)
Sales_Revenue_MF_Fprediction.to_pickle('Sales_Revenue_MF_Fprediction.pickle')

In [177]:
spf

,Client,Sales_MF_Prediction
46,285,1
81,1545,1
89,703,1
170,1195,1
269,1074,1
296,680,1
311,735,1
318,1503,1
332,1095,1
362,830,1
